# Doing data prepossessing for selecting best model

In [ ]:
import numpy as np
import pandas as pd
Data_train = pd.read_csv("train.csv")
Data_test  = pd.read_csv("test.csv")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MaxAbsScaler

X_train, X_test, y_train, y_test = train_test_split(Data_train.iloc[:,1:6], Data_train.iloc[:,6],random_state = 0, stratify = Data_train.iloc[:,6])


X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, random_state = 0, stratify = y_train)
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer(decode_error="ignore",ngram_range=(1, 3))

X_train_title_bigram = vectorizer.fit_transform(X_train.title)
X_test_title_bigram  = vectorizer.transform(X_test.title)

X_train_abstract_bigram = vectorizer.fit_transform(X_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(X_test.abstract)

vectorizer = TfidfVectorizer(decode_error="ignore",ngram_range=(1, 2))
X_train.references = X_train.references.fillna("")
X_test.references = X_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(X_train.references)
X_test_references_bigram  = vectorizer.transform(X_test.references)

from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()
X_train_year_onehot = encode.fit_transform(np.array(X_train.iloc[:,2]).reshape(-1,1))
X_test_year_onehot = encode.transform(np.array(X_test.iloc[:,2]).reshape(-1,1))

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)

X_train_year = poly.fit_transform(np.array(X_train.iloc[:,2]).reshape(-1,1))
X_test_year  = poly.transform(np.array(X_test.iloc[:,2]).reshape(-1,1))

X_train_n_citation = poly.fit_transform(np.array(X_train.iloc[:,4]).reshape(-1,1))
X_test_n_citation  = poly.transform(np.array(X_test.iloc[:,4]).reshape(-1,1))
from scipy.sparse import coo_matrix, hstack
X_train_bigram_numeric = hstack([X_train_title_bigram, 
                                 X_train_abstract_bigram, 
                                 X_train_references_bigram, 
                                 X_train_year_onehot, 
                                 X_train_year,
                                 X_train_n_citation])
X_test_bigram_numeric  = hstack([X_test_title_bigram, 
                                 X_test_abstract_bigram, 
                                 X_test_references_bigram, 
                                 X_test_year_onehot, 
                                 X_test_year,
                                 X_test_n_citation])

scaler = MaxAbsScaler()
X_train_bigram_numeric_scaled = scaler.fit_transform(X_train_bigram_numeric)
X_test_bigram_numeric_scaled  = scaler.transform(X_test_bigram_numeric)




# Doing the same to the whole train and test dataset

In [ ]:
vectorizer = TfidfVectorizer(decode_error="ignore",ngram_range=(1, 2))
X_train_title_bigram = vectorizer.fit_transform(Data_train.title)
X_test_title_bigram  = vectorizer.transform(Data_test.title)
X_train_abstract_bigram = vectorizer.fit_transform(Data_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(Data_test.abstract)
Data_train.references = Data_train.references.fillna("")
Data_test.references = Data_test.references.fillna("")


X_train_references_bigram = vectorizer.fit_transform(Data_train.references)
X_test_references_bigram  = vectorizer.transform(Data_test.references)

from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()
X_train_year_onehot = encode.fit_transform(np.array(Data_train.year).reshape(-1,1))
X_test_year_onehot = encode.transform(np.array(Data_test.year).reshape(-1,1))

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)

X_train_year = poly.fit_transform(np.array(Data_train.year).reshape(-1,1))
X_test_year  = poly.transform(np.array(Data_test.year).reshape(-1,1))

X_train_n_citation = poly.fit_transform(np.array(Data_train.n_citation).reshape(-1,1))
X_test_n_citation  = poly.transform(np.array(Data_test.n_citation).reshape(-1,1))


from scipy.sparse import hstack
real_X_train_bigram_numeric = hstack([X_train_title_bigram, 
                                      X_train_abstract_bigram, 
                                      X_train_references_bigram, 
                                      X_train_year_onehot,
                                      X_train_year,
                                      X_train_n_citation])
real_X_test_bigram_numeric  = hstack([X_test_title_bigram, 
                                      X_test_abstract_bigram, 
                                      X_test_references_bigram, 
                                      X_test_year_onehot,
                                      X_test_year,
                                      X_test_n_citation])


scaler = MaxAbsScaler()
real_X_train_bigram_numeric_scaled = scaler.fit_transform(real_X_train_bigram_numeric)
real_X_test_bigram_numeric_scaled  = scaler.transform(real_X_test_bigram_numeric)

# Fit the selected model onto the whole train dataset
I only put the final hyperparameter results here because the hyperparameter tuning process is messy, please contact me if it's anyhow needed.

## Code for prediction1104_v1_ensemble.csv

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
# import xgboost as xgb
estimators = []
from sklearn.neural_network import MLPClassifier
model4_1 = MLPClassifier(hidden_layer_sizes=(10), 
                       verbose = True,random_state = 0)
estimators.append(('NN1', model4_1))
model4_2 = MLPClassifier(hidden_layer_sizes=(11), 
                       verbose = True,random_state = 0)
estimators.append(('NN2', model4_2))
model4_3 = MLPClassifier(hidden_layer_sizes=(20), 
                       verbose = True,random_state = 0)
estimators.append(('NN3', model4_3))
# create the ensemble model
ensemble = VotingClassifier(estimators, n_jobs = -1)

ensemble.fit(X_train_bigram_numeric_scaled, y_train)
tr = ensemble.score(X_train_bigram_numeric_scaled, y_train)
te = ensemble.score(X_test_bigram_numeric_scaled, y_test)
print("train score = {}, test score = {}\n".format(tr,te))

In [ ]:
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1104_v1_ensemble.csv")
# The format of the output csv is weird, so I just manually change the column name of this csv.

## Code for prediction_13NN_ensemble.csv

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
# import xgboost as xgb
estimators = []
#model1 = LogisticRegression()
#estimators.append(('logistic', model1))
# model2 = MultinomialNB()
# estimators.append(('MulNB', model2))
# model2 = DecisionTreeClassifier()
# estimators.append(('cart', model2))
# model3 = LinearSVC(C = 0.012345444444444445)
# estimators.append(('svm', model3))
#model2 = LinearSVC(C = 1)
#estimators.append(('svm', model2))
#model3 = SGDClassifier(loss='hinge', penalty='l2',
                        #alpha=1e-2, random_state=42, max_iter=5, tol=None)
#estimators.append(('SGD', model3))
from sklearn.neural_network import MLPClassifier
estimators = []
model1 = MLPClassifier(hidden_layer_sizes=(10), 
                       verbose = True,random_state = 0)
estimators.append(('NN10', model1))
model2 = MLPClassifier(hidden_layer_sizes=(11), 
                       verbose = True,random_state = 0)
estimators.append(('NN11', model2))
model3 = MLPClassifier(hidden_layer_sizes=(14), 
                       verbose = True,random_state = 0)
estimators.append(('NN14', model3))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_10_11_14.csv")

estimators = []
model4 = MLPClassifier(hidden_layer_sizes=(15), 
                       verbose = True,random_state = 0)
estimators.append(('NN15', model4))
model5 = MLPClassifier(hidden_layer_sizes=(16), 
                       verbose = True,random_state = 0)
estimators.append(('NN16', model5))
model6 = MLPClassifier(hidden_layer_sizes=(18), 
                       verbose = True,random_state = 0)
estimators.append(('NN18', model6))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_15_16_18.csv")

estimators = []
model7 = MLPClassifier(hidden_layer_sizes=(24), 
                       verbose = True,random_state = 0)
estimators.append(('NN24', model7))
model8 = MLPClassifier(hidden_layer_sizes=(26), 
                       verbose = True,random_state = 0)
estimators.append(('NN26', model8))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_24_26.csv")

estimators = []
model9 = MLPClassifier(hidden_layer_sizes=(28), 
                       verbose = True,random_state = 0)
estimators.append(('NN28', model9))
model10 = MLPClassifier(hidden_layer_sizes=(29), 
                       verbose = True,random_state = 0)
estimators.append(('NN29', model10))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_28_29.csv")

estimators = []
model11 = MLPClassifier(hidden_layer_sizes=(31), 
                       verbose = True,random_state = 0)
estimators.append(('NN31', model11))
model12 = MLPClassifier(hidden_layer_sizes=(32), 
                       verbose = True,random_state = 0)
estimators.append(('NN32', model12))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_31_32.csv")


model13 = MLPClassifier(hidden_layer_sizes=(33), 
                       verbose = True,random_state = 0)
estimators.append(('NN33', model13))
model13.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(model13.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_33.csv")



In [ ]:
import pandas as pd
import os
import re

from scipy import stats

df = pd.DataFrame()
for file in os.listdir():
    r = re.compile(r'prediction1109')
    temp = r.search(file)
    if temp is not None:
        df['{}'.format(file)] = pd.read_csv(file).iloc[:,1]
    
output = df.apply(lambda x: stats.mode(x)[0][0], axis = 1)

output.name = 'Category'
output.to_csv('prediction_13NN_ensemble.csv')
# The format of the output csv is weird, so I just manually change the column name of this csv.